In [79]:
import numpy as np
import os
import pandas as pd 
import cv2
import imutils
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [47]:
classes = {"kirmizi_(red)":0,"sari_(yellow)":1,"yesil_(green)":2,"sol_kirmizi_(left_red)":3,"sag_kirmizi_(right_red)":4,"duz_kirmizi_(ahead_red)":5,"sol_sari_(left_yellow)":6,"sag_sari_(right_yellow)":7,"duz_sari_(ahead_yellow)":8,"sol_yesil_(left_green)":9,"sag_yesil_(right_green)":10,"duz_yesil_(ahead_green)":11}

In [48]:
def readDf(path):
    df = pd.read_csv(path)
    df = df.drop(columns='width')
    df = df.drop(columns='height')
    df = df.drop(columns='xmin')
    df = df.drop(columns='xmax')
    df = df.drop(columns='ymin')
    df = df.drop(columns='ymax')  
    df = df.drop(columns='class')
    df["name"] = [classes[item] for item in df['name']]
    return df

In [49]:
test_path = 'traffic_lights_dataset_v1_type1/test'
train_path = 'traffic_lights_dataset_v1_type1/train'
trainDf = readDf('traffic_lights_dataset_v1_type1/train.csv')
testDf = readDf('traffic_lights_dataset_v1_type1/test.csv')

In [50]:
train_image_paths = []
train_imageId = dict()
for image in os.listdir(train_path):
    if (str(image)[-3:] == 'jpg'):
        train_imageId[str(image)] = len(train_image_paths)
        train_image_paths.append(image)
train_image_paths
print(len(train_image_paths))

282


In [51]:
test_image_paths = []
test_imageId = dict()
for image in os.listdir(test_path):
    if (str(image)[-3:] == 'jpg'):
        test_imageId[str(image)] = len(test_image_paths)
        test_image_paths.append(image)
test_image_paths
print(test_imageId)
print(len(test_image_paths))

{'IMG_0919.jpg': 0, 'IMG_8390.jpg': 1, 'IMG_1082.jpg': 2, 'IMG_8617.jpg': 3, 'IMG_8630.jpg': 4, 'IMG_1024.jpg': 5, 'IMG_1076.jpg': 6, 'IMG_5749.jpg': 7, 'IMG_0915.jpg': 8, 'IMG_8403.jpg': 9, 'IMG_8557.jpg': 10, 'IMG_0920.jpg': 11, 'IMG_8971.jpg': 12, 'IMG_8667.jpg': 13, 'IMG_8605.jpg': 14, 'IMG_5698.jpg': 15, 'IMG_8527.jpg': 16, 'IMG_8666.jpg': 17, 'IMG_8665.jpg': 18, 'IMG_8364.jpg': 19, 'IMG_8947.jpg': 20, 'IMG_8868.jpg': 21, 'IMG_8363.jpg': 22, 'IMG_8528.jpg': 23, 'IMG_8362.jpg': 24, 'IMG_5349.jpg': 25, 'IMG_5402.jpg': 26, 'IMG_8529.jpg': 27, 'IMG_9007.jpg': 28, 'IMG_8946.jpg': 29, 'IMG_8595.jpg': 30, 'IMG_5391.jpg': 31, 'IMG_8945.jpg': 32, 'IMG_8493.jpg': 33, 'IMG_8391.jpg': 34, 'IMG_8631.jpg': 35, 'IMG_0970.jpg': 36, 'IMG_8491.jpg': 37, 'IMG_8558.jpg': 38, 'IMG_8402.jpg': 39, 'IMG_9006.jpg': 40, 'IMG_8404.jpg': 41, 'IMG_5287.jpg': 42, 'IMG_5350.jpg': 43, 'IMG_8869.jpg': 44, 'IMG_5694.jpg': 45, 'IMG_8604.jpg': 46, 'IMG_8632.jpg': 47, 'IMG_5401.jpg': 48, 'IMG_8969.jpg': 49, 'IMG_8846

In [52]:
trainLabels = [ [0]*len(train_image_paths) for _ in range(12) ]
trainData = trainDf.values.tolist()
cnt = 0
for row  in trainData:
    trainLabels[row[1]][train_imageId[row[0]]] = 1

In [53]:
testLabels = [ [0] * len(test_image_paths) for _ in range(12) ]
testData = testDf.values.tolist()
cnt = 0
for row  in testData:
    testLabels[row[1]][test_imageId[row[0]]] = 1

In [54]:
def image_to_feature_vector(image, size=(32, 32)):
	return cv2.resize(image, size).flatten()

In [55]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	else:
		cv2.normalize(hist, hist)
	return hist.flatten()

In [56]:
rawImages = []
features = []
for image_path in train_image_paths:
	image = cv2.imread(train_path + '/' + image_path)
	pixels = image_to_feature_vector(image)
	hist = extract_color_histogram(image)

	rawImages.append(pixels)
	features.append(hist)

In [57]:
test_rawImages = []
test_features = []
for image_path in test_image_paths:
	image = cv2.imread(test_path + '/' + image_path)
	pixels = image_to_feature_vector(image)
	hist = extract_color_histogram(image)

	test_rawImages.append(pixels)
	test_features.append(hist)

In [58]:
rawImages = np.array(rawImages)
features = np.array(features)
# labs = np.array(trainLabels[2])

In [59]:
model = [0]*12
for i in range(12):
    model[i] = KNeighborsClassifier(n_neighbors=2,n_jobs=-1)
    model[i].fit(features, trainLabels[i])

In [80]:
# print(testLabels[1])
cumPred = []
cumLabel = []
for _ in range(len(test_imageId)):
	cumPred.append([])
	cumLabel.append([])

for i in range(12):
	predictions = model[i].predict(test_features).tolist()
	acc = model[i].score(test_features, testLabels[i])
	if (acc != 1):
		f1 = f1_score(testLabels[i],predictions)
	else:
		f1 = 1
	for j in range(len(predictions)):
		cumPred[j].append(predictions[j])
		cumLabel[j].append(testLabels[i][j])

	print('Acc for class ' + str(i) + " : " + str(acc))

print(accuracy_score(cumLabel,cumPred))
print(f1_score(cumLabel,cumPred,average='weighted'))
# print("Cumulative Acc : " + str(cumAcc))

Acc for class 0 : 0.7166666666666667
Acc for class 1 : 0.9666666666666667
Acc for class 2 : 0.8166666666666667
Acc for class 3 : 0.95
Acc for class 4 : 1.0
Acc for class 5 : 0.9666666666666667
Acc for class 6 : 1.0
Acc for class 7 : 1.0
Acc for class 8 : 1.0
Acc for class 9 : 1.0
Acc for class 10 : 1.0
Acc for class 11 : 1.0


TypeError: accuracy_score() got an unexpected keyword argument 'average'